<a href="https://colab.research.google.com/github/Prasanna0249/Smart-Text-summarizer/blob/main/0Text_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Installing required libraries
!pip install transformers newspaper3k ipywidgets lxml_html_clean --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.7 MB/s eta 0:00:00


In [2]:
# Step 1: Installing required libraries
!pip install transformers newspaper3k ipywidgets lxml_html_clean --quiet
# Step 2: Import necessary packages
from transformers import pipeline
from newspaper import Article
import ipywidgets as widgets
from IPython.display import display, HTML

In [3]:
# Step 3: Load the pre-trained summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [4]:
# Step 4: Helper functions

# Function to summarize text using HuggingFace's BART model
def summarize_text(text, max_len=130):
    result = summarizer(text, max_length=max_len, min_length=30, do_sample=False)
    return result[0]['summary_text']

# Function to extract content from a news/article URL
def extract_from_url(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except:
        return "Error fetching article content. Please check the URL."

# Function to handle uploaded .txt files
def read_uploaded_file(file_obj):
    content = file_obj[list(file_obj.keys())[0]]['content']
    return content.decode('utf-8')


In [5]:
# Step 5: Create interactive widgets

# Text area for manual input
text_input = widgets.Textarea(
    placeholder='Paste or type your text here...',
    description='Text:',
    layout=widgets.Layout(width='100%', height='150px')
)

# File uploader for .txt files
file_upload = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description='Upload .txt File'
)

# URL input for summarizing online articles
url_box = widgets.Text(
    placeholder='Paste article URL here...',
    description='URL:',
    layout=widgets.Layout(width='100%')
)

# Dropdown for choosing summary length
length_choice = widgets.Dropdown(
    options=[('Short', 60), ('Medium', 130), ('Long', 200)],
    value=130,
    description='Summary Length'
)

# Button to start summarization
summarize_button = widgets.Button(
    description='Generate Summary',
    button_style='success',
    tooltip='Click to generate summary'
)

# Output area for displaying results
output_box = widgets.Output()


In [6]:
# Step 6: Define what happens when the button is clicked

def on_click_generate_summary(btn):
    with output_box:
        output_box.clear_output()

        # Priority: URL > File > Text
        text = ""

        if url_box.value.strip():
            text = extract_from_url(url_box.value)
            if text.startswith("Error"):
                print(text)
                return
            print("Article extracted from URL.\n")

        elif file_upload.value:
            text = read_uploaded_file(file_upload.value)
            print("Text loaded from uploaded file.\n")

        elif text_input.value.strip():
            text = text_input.value
            print("Text received from input box.\n")

        else:
            print("Please provide input using text, file, or URL.")
            return

        # Generate summary
        summary = summarize_text(text, max_len=length_choice.value)

        # Word stats
        input_len = len(text.split())
        summary_len = len(summary.split())
        compression = round(summary_len / input_len, 2)

        # Display results
        display(HTML(f"""
        <h4>📝 Summary:</h4>
        <div style="background:#f9f9f9;padding:10px;border-left:5px solid #007BFF;">
        {summary}
        </div><br>
        <b>Original Text Length:</b> {input_len} words<br>
        <b>Summary Length:</b> {summary_len} words<br>
        <b>Compression Ratio:</b> {compression} (lower is better)
        """))

# Bind the button to the click function
summarize_button.on_click(on_click_generate_summary)


In [9]:
# Step 7: Display the complete interface in Colab

display(
    widgets.VBox([
        text_input,
        file_upload,
        url_box,
        length_choice,
        summarize_button,
        output_box
    ])
)
